download de pdf usando requests lendo pdf direto do buffer da memoria

In [1]:
# file = requests.get('https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta=2019|09_Setembro|DODF%20185%2027-09-2019|&arquivo=DODF%20185%2027-09-2019%20INTEGRA.pdf')

# with open('novo_test.pdf','wb') as arqv:
#     arqv.write(file.content)

# file = requests.get('https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta=2019|09_Setembro|DODF%20185%2027-09-2019|&arquivo=DODF%20185%2027-09-2019%20INTEGRA.pdf')
# pdf = PyPDF2.PdfFileReader(io.BytesIO(file.content))
# pdf.numPages

In [2]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
import io
import re
from multiprocessing.pool import ThreadPool
import pandas as pd

In [3]:
base_url = "https://www.dodf.df.gov.br/listar"

In [4]:
import requests

cookies = {
    '_ga': 'GA1.4.1676642850.1569274904',
    '_gid': 'GA1.4.30087383.1569274904',
    'PHPSESSID': 'c8gp0cncq0bshqvnkq5ng3d0g7',
}

headers = {
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Origin': 'https://www.dodf.df.gov.br',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en,en-US;q=0.9,pt-BR;q=0.8,pt;q=0.7',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
    'Accept': '*/*',
    'Referer': 'https://www.dodf.df.gov.br/listar',
    'X-Requested-With': 'XMLHttpRequest',
    'Connection': 'keep-alive',
    'Content-Length': '0',
}

In [30]:
def get_url_to_request(string):
    st_p, nd_p, rd_p, th_p = \
    re.sub(pattern=r"\s", repl="%20",string=string).split('/')
    return th_p, "https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta="+st_p+"|"+nd_p+"|"+rd_p+"|&arquivo="+th_p

In [26]:
def get_file(th_param, url):
    file = requests.get(url)
    with open(th_param,'wb') as arqv:
        arqv.write(file.content)

In [ ]:
params = (('dir', '/'),)
response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
years = json.loads(response.text)['data'][:4]
years = list(map(lambda x:str(x),years))

all_docs = []
for year in years:
    params = (('dir', year),)
    response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
    months = json.loads(response.text)['data']
    for month in months:
        params = (('dir', year+'/'+month),)
        response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
        docs = json.loads(response.text)['data']
        docs = list(docs.values())
        for doc in docs:
            params = (('dir', year+'/'+month+'/'+doc),)
            response = requests.post(base_url, headers=headers, params=params, cookies=cookies)
            all_docs.append(response.json()['data'][0])

In [ ]:
#all_docs = [re.sub(pattern=r"(MarÃ§o)", repl="Março", string=doc) for doc in all_docs]

In [178]:
#all_docs = pd.DataFrame(all_docs)

In [ ]:
#all_docs.to_csv('all_documents.csv',index=False)

In [21]:
with open('all_documents.csv','r') as file:
    all_docs = file.readlines()

In [22]:
all_docs = [re.sub(pattern=r"(\n)", repl="", string=doc) for doc in all_docs]

In [ ]:
for doc in ThreadPool(8).map(get_url_to_request, all_docs):
    print(doc)

In [33]:
get_file('DODF%20185%2027-09-2019%20INTEGRA.pdf', 'https://www.dodf.df.gov.br/index/visualizar-arquivo/?pasta=2019|09_Setembro|DODF%20185%2027-09-2019|&arquivo=DODF%20185%2027-09-2019%20INTEGRA.pdf')